# Análisis de datos para la creación de un modelo de estimación de niveles de obesidad basado en hábitos alimenticios y de condición física

Iniciaremos la exploración de este [conjunto de datos](https://archive.ics.uci.edu/ml/datasets/Estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition+) encontrado en el repositorio de Machine Learning de la [UCI](https://archive.ics.uci.edu/ml/index.html) recabado y donado por:
- Fabio Mendoza Palechor, Email: fmendoza1@cuc.edu.co, Telefono: +573182929611
- Alexis de la Hoz Manotas, Email: akdelahoz@gmail.com, Telefono: +573017756983

El conjunto incluye datos sobre la estimación de niveles de obesidad de personas de paises como México, Perú y Colombia, basados en sus hábitos alimenticios y su condición física.

### Características:
|Característica|Nombre en el Dataset|Unidad|
|:-------------|:-------------------|:----:|
|Género |Gender    | Female, Male|
|Edad   |Age | int (años)|
|Peso   | Weight | int  (kg) |
|Talla  | Height | float (mts) |
|Familiar con sobrepeso| family_history_with_overweight | yes, no |
|Come frecuentemente alimentos con altos niveles calóricos|FAVC|yes, no|
|Come vegetales en sus comidas|FCVC|1: Nunca, 2: Algunas veces, 3: Siempre|
|Número de comidas principales al día|NPC|1: Entre 1 y 2, 2: Tres, 3: Más de tres|
|Come algún alimento entre comidas|CAEC| no, Sometimes, Frequently, Always|
|Fumador|SMOKE| yes, no|
|Consumo de agua diario|CH20|1: Menos de un litro, 2: Entre 1 y 2 litros, 3: Más de 2 litros|
|Monitorea el número de calorías que consume diario|SCC|yes, no|
|Frecuencia con la que practica actividad física|FAF|0: No practica, 1: 1 o 2 días, 2: 2 o 4 días, 3: 4 o 5 días|
|Tiempo que usa dispositivos electrónicos al día|TUE|0: 0-2 horas, 1: 3-5 horas, 2: Más de 5 horas|
|Consumo de alcohol|CALC|no, Sometimes, Frequently, Always|
|Medio de transporte principal|MTRANS|Automobile, Motorbike, Bike, Public_Transportation, Walking|
|Nivel de obesidad (Variable objetivo)|NObeyesdad|Insufficient_Weight, Normal_Weight, Overweight_Level_I, Overweight_Level_II, Obesity_Type_I, Obesity_Type_II, Obesity_Type_III

## Conociendo y limpiando los datos

In [1]:
# importando librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# importando el dataset
# en este caso el carácter que delimita es ','
data = pd.read_csv("ObesityDataSet.csv", delimiter = ',')
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
# Identificando la presencia de valores nulos
data.isna().sum()
# En este caso, no hay

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [4]:
# como se puede notar hay muchos atributos con campos categóricos, se realizará el etiquetado de estos campos
# para poder usarlos en los modelos a entrenar
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

columns_to_encode = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']

for encode in columns_to_encode:
    data[encode] = labelencoder.fit_transform(data[encode])
    print(dict(zip(labelencoder.classes_, labelencoder.transform(labelencoder.classes_))))

{'Female': 0, 'Male': 1}
{'no': 0, 'yes': 1}
{'no': 0, 'yes': 1}
{'Always': 0, 'Frequently': 1, 'Sometimes': 2, 'no': 3}
{'no': 0, 'yes': 1}
{'no': 0, 'yes': 1}
{'Always': 0, 'Frequently': 1, 'Sometimes': 2, 'no': 3}
{'Automobile': 0, 'Bike': 1, 'Motorbike': 2, 'Public_Transportation': 3, 'Walking': 4}
{'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Obesity_Type_I': 2, 'Obesity_Type_II': 3, 'Obesity_Type_III': 4, 'Overweight_Level_I': 5, 'Overweight_Level_II': 6}


In [5]:
# ahora indexaremos los demás atributos para que empiezen desde 0 en vez de 1

columns_to_reindex = ["FCVC","NCP","CH2O"]

for col in columns_to_reindex:
    data[col] = data[col]-1

### Nueva indexación de las características
|Característica|Nombre en el Dataset|Unidad|
|:-------------|:-------------------|:----:|
|Género |Gender    | Female, Male|
|Edad   |Age | int (años)|
|Peso   | Weight | int  (kg) |
|Talla  | Height | float (mts) |
|Familiar con sobrepeso| family_history_with_overweight | 0: no, 1: yes |
|Come frecuentemente alimentos con altos niveles calóricos|FAVC| 0: no, 1: yes |
|Come vegetales en sus comidas|FCVC|0: Nunca, 1: Algunas veces, 2: Siempre|
|Número de comidas principales al día|NPC|0: Entre 1 y 2, 1: Tres, 2: Más de tres|
|Come algún alimento entre comidas|CAEC|0: Always, 1: Frequently, 2: Sometimes, 3: no|
|Fumador|SMOKE| 0: no, 1: yes |
|Consumo de agua diario|CH20|0: Menos de un litro, 1: Entre 1 y 2 litros, 2: Más de 2 litros|
|Monitorea el número de calorías que consume diario|SCC| 0: no, 1: yes |
|Frecuencia con la que practica actividad física|FAF|0: No practica, 1: 1 o 2 días, 2: 2 o 4 días, 3: 4 o 5 días|
|Tiempo que usa dispositivos electrónicos al día|TUE|0: 0-2 horas, 1: 3-5 horas, 2: Más de 5 horas|
|Consumo de alcohol|CALC|0: Always, 1: Frequently, 2: Sometimes, 3: no|
|Medio de transporte principal|MTRANS|0: Automobile, 1: Motorbike, 2: Bike, 3: Public_Transportation, 4: Walking|
|Nivel de obesidad (Variable objetivo)|NObeyesdad|0: Insufficient_Weight, 1: Normal_Weight, 2: Overweight_Level_I, 3: Overweight_Level_II, 4: Obesity_Type_I, 5: Obesity_Type_II, 6: Obesity_Type_III|

In [6]:
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.0,1.62,64.0,1,0,1.0,2.0,2,0,1.0,0,0.0,1.0,3,3,1
1,0,21.0,1.52,56.0,1,0,2.0,2.0,2,1,2.0,1,3.0,0.0,2,3,1
2,1,23.0,1.80,77.0,1,0,1.0,2.0,2,0,1.0,0,2.0,1.0,1,3,1
3,1,27.0,1.80,87.0,0,0,2.0,2.0,2,0,1.0,0,2.0,0.0,1,4,5
4,1,22.0,1.78,89.8,0,0,1.0,0.0,2,0,1.0,0,0.0,0.0,2,3,6


In [7]:
# Identificando filas duplicadas
duplicados = len(data) - len(data.drop_duplicates())  # Solo para ver cuantas filas estan duplicadas
print("filas duplicadas = {0}".format(duplicados)) 

# En este caso no hace falta eliminar filas duplicadas, debido a la naturaleza de los datos
# data.drop_duplicates(inplace=True)
# print("{0} campos duplicados borrados, nuevas dimensiones del dataset = {1}".format(duplicados,data.shape))

filas duplicadas = 24


In [8]:
# Ahora con los datos un poco más legibles podemos ver un resumen rápido de las características
data.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Gender,2111.0,0.51,0.50,0.00,0.00,1.00,1.00,1.00
Age,2111.0,24.31,6.35,14.00,19.95,22.78,26.00,61.00
Height,2111.0,1.70,0.09,1.45,1.63,1.70,1.77,1.98
Weight,2111.0,86.59,26.19,39.00,65.47,83.00,107.43,173.00
family_history_with_overweight,2111.0,0.82,0.39,0.00,1.00,1.00,1.00,1.00
FAVC,2111.0,0.88,0.32,0.00,1.00,1.00,1.00,1.00
FCVC,2111.0,1.42,0.53,0.00,1.00,1.39,2.00,2.00
NCP,2111.0,1.69,0.78,0.00,1.66,2.00,2.00,3.00
CAEC,2111.0,1.86,0.47,0.00,2.00,2.00,2.00,3.00
SMOKE,2111.0,0.02,0.14,0.00,0.00,0.00,0.00,1.00


## Comparando diferentes modelos predictivos
<p>Lo primero será separar nuestra variable objetivo del resto de los datos.</p>

In [9]:
# separamos la variable objetivo del resto de los datos
x = data.drop(['NObeyesdad'], axis=1,inplace=False)
y = data['NObeyesdad']

# Separamos el conjunto de datos de entrenamiento de el de pruebas (80% y 20% respectivamente)
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

### Probando con algunos modelos predictivos de sckit-learn
Entrenaremos varios modelos de aprendizaje máquina clásicos en busqueda de uno que se ajuste a los datos con un procentaje de exactitud aceptable, es importante también tener en cuenta que debemos evitar un sobreajuste a la hora de entrenar los modelos, es por esto último que usaremos la técnica de validación cruzada con 5 divisiones.

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

dec = DecisionTreeClassifier()
ran = RandomForestClassifier(n_estimators=100)
knn = KNeighborsClassifier(n_neighbors=100)
svm = SVC(random_state=1)
naive = GaussianNB()

models = {"Decision tree" : dec,
          "Random forest" : ran,
          "KNN" : knn,
          "SVM" : svm,
          "Naive bayes" : naive}

scores= { }

for key, value in models.items():    
    model = value
    model.fit(x_train, y_train)
    #scores[key] = model.score(x_test, y_test)
    score = cross_val_score(model,x_test,y_test,cv=5)
    scores[key] = ("%0.2f accuracy with a standard deviation of %0.2f" % (score.mean(), score.std()))

# mostrando resultados
scores_frame = pd.DataFrame(scores, index=["Accuracy Score"]).T
scores_frame.sort_values(by=["Accuracy Score"], axis=0 ,ascending=False, inplace=True)
scores_frame

,Accuracy Score
Random forest,0.88 accuracy with a standard deviation of 0.02
Decision tree,0.85 accuracy with a standard deviation of 0.05
SVM,0.54 accuracy with a standard deviation of 0.03
Naive bayes,0.48 accuracy with a standard deviation of 0.02
KNN,0.47 accuracy with a standard deviation of 0.05


In [11]:
# guardamos el modelo con Joblib de SciPy
import joblib
nombre_archivo = "obsesidad_nutridesk_v1.model"
joblib.dump(models['Random forest'], nombre_archivo)

['obsesidad_nutridesk_v1.model']

In [12]:
# después para hacer una predicción solo volmenos a cargar el modelo
modelo_cargado = joblib.load(nombre_archivo)
result = modelo_cargado.score(x_test, y_test)
print(result)

0.950354609929078


In [21]:
# vamos a probar el modelo con datos de prueba
prueba =[[1,23.9,1.74,68,1,0,3,2,2,0,2,0,1,1,0,3]]
p_data = pd.DataFrame(data=prueba, columns = ['Gender','Age','Height','Weight','family_history_with_overweight',
                                              'FAVC','FCVC','NPC','CAEC','SMOKE',
                                              'CH2O','SCC','FAF','TUE','CALC','MTRANS'])

# esta persona es sana en la realidad, por lo tanto la predicción es correcta
modelo_cargado.predict(p_data) 

array([1])